In [1]:
#import necessary modules
import os
import sys
import pandas as pd

In [4]:
file_path = "/home/ltgwgeorge/Desktop/GeorgeCode/python/pythonGeorge/LearnMachineLearning/datasets/Churn.csv"
df = pd.read_csv(file_path)
df.head()

,Customer ID,Gender,Senior Citizen,Partner,Dependents,tenure,Phone Service,Multiple Lines,Internet Service,Online Security,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEA,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
2,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
4,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. Drop unnecessary columns
df = df.drop(columns=["Customer ID"])

# 2. Convert 'Total Charges' to numeric (some values may be empty or invalid)
df["Total Charges"] = pd.to_numeric(df["Total Charges"], errors="coerce")

# 3. Handle missing values (if any exist after conversion)
df.dropna(inplace=True)

# 4. Convert 'Churn' column to binary (Yes → 1, No → 0)
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# 5. One-Hot Encode categorical columns (automatically drops first to avoid multicollinearity)
df_encoded = pd.get_dummies(df, drop_first=True)

# 6. Split into features and labels
X = df_encoded.drop("Churn", axis=1)
y = df_encoded["Churn"]

# 7. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8. Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Show result
X_train_scaled[:2], y_train[:2].values


(array([[-0.43956018, -0.25865123, -0.28862469, -0.32013934, -1.0096448 ,
         -0.96300176, -0.64585483,  0.32717651, -0.32717651, -0.85752466,
         -0.88116093, -0.52510662, -0.52510662, -0.63261292, -0.52510662,
          1.38166454, -0.52510662,  1.38275388, -0.52510662, -0.63729778,
         -0.52510662, -0.78942813, -0.52510662, -0.79984697,  1.94212269,
         -0.55750409,  0.82669408, -0.52483116, -0.71807033,  1.84637236],
        [ 2.27500136,  0.10807554,  1.53251632,  0.6877746 , -1.0096448 ,
         -0.96300176, -0.64585483,  0.32717651, -0.32717651,  1.16614722,
          1.13486648, -0.52510662, -0.52510662,  1.58074546, -0.52510662,
          1.38166454, -0.52510662,  1.38275388, -0.52510662,  1.56912518,
         -0.52510662,  1.26673976, -0.52510662,  1.25023916,  1.94212269,
         -0.55750409,  0.82669408, -0.52483116, -0.71807033, -0.54160256]]),
 array([0, 0]))

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the neural network model
model = Sequential([
    Dense(32, input_shape=(X_train_scaled.shape[1],), activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=20, batch_size=32)

# Evaluate performance
loss, accuracy = model.evaluate(X_test_scaled, y_test)
loss, accuracy


2025-06-20 09:11:20.385720: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 09:11:20.524843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750399880.581277   11417 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750399880.598465   11417 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750399880.719034   11417 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6605 - loss: 0.5739 - val_accuracy: 0.7797 - val_loss: 0.4560
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8053 - loss: 0.4311 - val_accuracy: 0.7861 - val_loss: 0.4425
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8045 - loss: 0.4223 - val_accuracy: 0.7896 - val_loss: 0.4396
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8069 - loss: 0.4088 - val_accuracy: 0.7882 - val_loss: 0.4371
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8059 - loss: 0.4248 - val_accuracy: 0.7861 - val_loss: 0.4392
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8092 - loss: 0.4039 - val_accuracy: 0.7910 - val_loss: 0.4341
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8216 - loss: 0.3974 - val_accuracy: 0.7903 - val_loss: 0.4332
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8196 - loss: 0.3991 - val_accuracy: 0.

(0.446517676115036, 0.7860696315765381)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the model
model = Sequential([
    Dense(32, input_shape=(X_train_scaled.shape[1],), activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7005 - loss: 0.5494 - val_accuracy: 0.7846 - val_loss: 0.4399
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8090 - loss: 0.4190 - val_accuracy: 0.7854 - val_loss: 0.4316
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7989 - loss: 0.4204 - val_accuracy: 0.7818 - val_loss: 0.4310
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8126 - loss: 0.4014 - val_accuracy: 0.7818 - val_loss: 0.4315
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8048 - loss: 0.4086 - val_accuracy: 0.7861 - val_loss: 0.4294
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8088 - loss: 0.4005 - val_accuracy: 0.7825 - val_loss: 0.4312
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8193 - loss: 0.4038 - val_accuracy: 0.7811 - val_loss: 0.4304
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8054 - loss: 0.4035 - val_accuracy: 0.

In [1]:
# Save the entire model to a folder
model.save("churn_model")


NameError: name 'model' is not defined